# İçerik tabanlı kitap tavsiye NLP yaklaşımı

In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,başlık,tür,yazar,konu
0,1,Anne Karenina,"Aşk, Dram",Tolstoy,Aşkı yüzlerce yazar tarafından ele alınmış bir...
1,2,Aşk ve Gurur,"Aşk, Dram",Jane Austen,Orta halli bir ailenin zeki ve neşeli kızı ile...
2,3,Yabancı,"Felsefe, Eleştiri",Albert Camus,Kitabın olay örgüsünde her şey çok kısa bir za...
3,4,Böyle Buyurdu Zerdüşt,"Felsefe, Eleştiri",Nietzsche,dâhi Alman filozof Friedrich Nietzsche'nin fel...
4,5,Küçük Prens,"Eleştiri, Kurgu",Antoine de Saint-Exupéry,Gezegenindeki çiçeğiyle pek anlaşamadığı için ...


In [2]:
df.shape

(20, 5)

In [3]:
df = df[['başlık','tür','yazar','konu']]
df.head()

,başlık,tür,yazar,konu
0,Anne Karenina,"Aşk, Dram",Tolstoy,Aşkı yüzlerce yazar tarafından ele alınmış bir...
1,Aşk ve Gurur,"Aşk, Dram",Jane Austen,Orta halli bir ailenin zeki ve neşeli kızı ile...
2,Yabancı,"Felsefe, Eleştiri",Albert Camus,Kitabın olay örgüsünde her şey çok kısa bir za...
3,Böyle Buyurdu Zerdüşt,"Felsefe, Eleştiri",Nietzsche,dâhi Alman filozof Friedrich Nietzsche'nin fel...
4,Küçük Prens,"Eleştiri, Kurgu",Antoine de Saint-Exupéry,Gezegenindeki çiçeğiyle pek anlaşamadığı için ...


In [4]:
df.shape

(20, 4)

In [5]:
df['tür'] = df['tür'].map(lambda x: x.lower().split(','))
df['yazar'] = df['yazar'].map(lambda x: x.split(' '))

for index, row in df.iterrows():
    row['yazar'] = ''.join(row['yazar']).lower()

In [6]:
df['anahtar'] = ""

for index, row in df.iterrows():
    konu = row['konu']
    
    r = Rake()

    r.extract_keywords_from_text(konu)
    anahtar_skor = r.get_word_degrees()
    
    row['anahtar'] = list(anahtar_skor.keys())

df.drop(columns = ['konu'], inplace = True)

In [7]:
df.set_index('başlık', inplace = True)
df.head()

,tür,yazar,anahtar
başlık,,,
Anne Karenina,"[aşk, dram]",tolstoy,"[19, yüzyıldaki, rus, aristokrasisinde, yaşana..."
Aşk ve Gurur,"[aşk, dram]",janeausten,"[mağrur, olmasının, yanı, sıra, son, derece, d..."
Yabancı,"[felsefe, eleştiri]",albertcamus,"[kendisini, adım, ölüme, götüren, süreci, tama..."
Böyle Buyurdu Zerdüşt,"[felsefe, eleştiri]",nietzsche,"[dâhi, alman, filozof, friedrich, nietzsche, k..."
Küçük Prens,"[eleştiri, kurgu]",antoinedesaint-exupéry,"[kimdir, küçük, prens, neler, öğrenmiştir, gez..."


In [8]:
df['anahtarlar'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'yazar':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['anahtarlar'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'anahtarlar'], inplace = True)

In [9]:
df.head()

,anahtarlar
başlık,
Anne Karenina,aşk dram tolstoy 19 yüzyıldaki rus aristokras...
Aşk ve Gurur,aşk dram janeausten mağrur olmasının yanı sır...
Yabancı,felsefe eleştiri albertcamus kendisini adım ö...
Böyle Buyurdu Zerdüşt,felsefe eleştiri nietzsche dâhi alman filozof...
Küçük Prens,eleştiri kurgu antoinedesaint-exupéry kimdir ...


In [10]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['anahtarlar'])

indices = pd.Series(df.index)
indices[:5]

0            Anne Karenina
1             Aşk ve Gurur
2                  Yabancı
3    Böyle Buyurdu Zerdüşt
4              Küçük Prens
Name: başlık, dtype: object

In [11]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [16]:
def tavsiye(baslik, cosine_sim = cosine_sim):
    tavsiyeler = []
    idx = indices[indices == baslik].index[0]
    skorlar = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    print(skorlar)
    top10 = list(skorlar.iloc[1:6].index)
    for i in top10:
        tavsiyeler.append(list(df.index)[i])
    return tavsiyeler

In [20]:
tavsiye('Hayvan Çiftliği')

11    1.000000
12    0.204465
6     0.180540
4     0.120096
13    0.118864
10    0.108786
3     0.099627
14    0.083624
2     0.081786
19    0.080911
15    0.068279
17    0.057213
0     0.046225
7     0.044412
1     0.038837
8     0.034401
18    0.033884
9     0.031404
16    0.025863
5     0.025008
dtype: float64


['Otomatik Portakal',
 '1984',
 'Küçük Prens',
 'Cesur Yeni Dünya',
 'Fahrenheit 451']

In [111]:
def tavsiye_cift(birinci, ikinci, cosine_sim = cosine_sim):
    tavsiyeler = []
    idx = indices[indices == birinci].index[0]
    skorlar1 = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    s1 = skorlar1.iloc[1:,]
    idx = indices[indices == ikinci].index[0]
    skorlar2 = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    s2 = skorlar2.iloc[1:,]
    sonuc = s1.combine(s2, func=(lambda x1, x2: (x1 + x2)/2)).sort_values(ascending = False)
    top5 = list(sonuc.iloc[1:6].index)
    for i in top5:
        tavsiyeler.append(list(df.index)[i])
    x=0
    print("{} ve {} kitapları baz alınarak yapılan ilk 5 tavsiye:".format(birinci,ikinci))
    for i in tavsiyeler:
        x+=1
        print("{}. {}".format(x,i))

In [113]:
tavsiye_cift("Siddhartha","Cesur Yeni Dünya")

Siddhartha ve Cesur Yeni Dünya kitapları baz alınarak yapılan ilk 5 tavsiye:
1. Fahrenheit 451
2. Suç ve Ceza
3. Dönüşüm
4. Sineklerin Tanrısı
5. Böyle Buyurdu Zerdüşt
